In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
import torch
from tqdm import tqdm

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
file_path = 'train.csv'
data = pd.read_csv(file_path)

# Data Preprocessing
X = data['sentence'].tolist()
y = pd.factorize(data['label'])[0]  # Convert labels to numerical format
label_to_index = {label: i for i, label in enumerate(data['label'].unique())}
index_to_label = {i: label for label, i in label_to_index.items()}

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Dataset class
class IntentDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len=128):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            sentence,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Prepare datasets and loaders
train_dataset = IntentDataset(X_train, y_train, tokenizer)
val_dataset = IntentDataset(X_val, y_val, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Load pre-trained BERT model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(label_to_index))
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training and Evaluation Loop
def train_epoch(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    correct_predictions = 0

    for batch in tqdm(data_loader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        total_loss += loss.item()

        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)

        loss.backward()
        optimizer.step()

    return total_loss / len(data_loader), correct_predictions.double() / len(data_loader.dataset)

def evaluate_epoch(model, data_loader, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits
            total_loss += loss.item()

            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return total_loss / len(data_loader), correct_predictions.double() / len(data_loader.dataset), predictions, true_labels

# Train the model
EPOCHS = 10
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, device)
    val_loss, val_acc, val_preds, val_labels = evaluate_epoch(model, val_loader, device)

    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

# Save Model
model.save_pretrained('./roberta_intent_model')
tokenizer.save_pretrained('./roberta_intent_model')


c:\Users\pradi\OneDrive\Desktop\Tifin_Test_1\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\pradi\OneDrive\Desktop\Tifin_Test_1\.conda\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Epoch 1/10


Evaluation: 100%|██████████| 5/5 [00:00<00:00, 21.86it/s]


Train Loss: 3.0122, Train Accuracy: 0.0649
Validation Loss: 2.9105, Validation Accuracy: 0.1212

Epoch 2/10


Evaluation: 100%|██████████| 5/5 [00:00<00:00, 21.34it/s]


...

Evaluation: 100%|██████████| 5/5 [00:00<00:00, 20.35it/s]


Train Loss: 0.0746, Train Accuracy: 1.0000
Validation Loss: 0.4795, Validation Accuracy: 0.9091

Epoch 10/10


Evaluation: 100%|██████████| 5/5 [00:00<00:00, 21.32it/s]


Train Loss: 0.0572, Train Accuracy: 1.0000
Validation Loss: 0.4694, Validation Accuracy: 0.9242


('./bert_intent_model\\tokenizer_config.json',
 './bert_intent_model\\special_tokens_map.json',
 './bert_intent_model\\vocab.json',
 './bert_intent_model\\merges.txt',
 './bert_intent_model\\added_tokens.json')

In [2]:

# Classification Report
print("\nClassification Report:")
print(classification_report(val_labels, val_preds, target_names=list(label_to_index.keys())))



Classification Report:
                       precision    recall  f1-score   support

                  EMI       0.83      1.00      0.91         5
                  COD       1.00      1.00      1.00         2
       ORTHO_FEATURES       1.00      1.00      1.00         3
        ERGO_FEATURES       1.00      1.00      1.00         4
           COMPARISON       1.00      1.00      1.00         1
             WARRANTY       1.00      1.00      1.00         5
100_NIGHT_TRIAL_OFFER       1.00      0.75      0.86         4
   SIZE_CUSTOMIZATION       1.00      1.00      1.00         1
   WHAT_SIZE_TO_ORDER       1.00      0.75      0.86         4
             LEAD_GEN       0.80      1.00      0.89         4
        CHECK_PINCODE       1.00      1.00      1.00         1
         DISTRIBUTORS       1.00      0.88      0.93         8
        MATTRESS_COST       1.00      1.00      1.00         3
     PRODUCT_VARIANTS       0.67      1.00      0.80         2
   ABOUT_SOF_MATTRESS       1.